# Pre-Procesamiento para NLP mediante TF-IDF

Este notebook será la plantilla que se utilizará para realizar el pre - procesamiento de los datos y darle uso a TF-IDF. con el fin de utilizar la base final como base de los siguientes modelos en el desarrollo del ejercicio:

## Obtención de los datos

Comenzaremos con traer los datos necesarios e importar los paquetes requeridos para el desarrollo del ejercicio:

In [2]:
# Loads Constants, imports and utilitary functions
%run 0_Utils.ipynb

importamos los paquetes que utilizaremos para el procesamiento de los datos:

In [3]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re

#lemmatizer = WordNetLemmatizer()
#stemmer = PorterStemmer() 

In [4]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1125)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1125)>


False

In [ ]:
# Esta celda solo debe ser usada en caso de que la celda anterior falle y no sea posible descargar los paquetes de NLTK.
# https://stackoverflow.com/questions/38916452/nltk-download-ssl-certificate-verify-failed

#import nltk
#import ssl

#try:
#    _create_unverified_https_context = ssl._create_unverified_context
#except AttributeError:
#    pass
#else:
#    ssl._create_default_https_context = _create_unverified_https_context

#nltk.download()

In [5]:
df = getLatestDataset()
df.shape

<ipython-input-2-3fec7da56bed>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_facturas['Categoria'] = df_facturas['issuer'].map(diccionario)
<ipython-input-2-3fec7da56bed>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_facturas['Categoria'] = "No factura"


(1783, 9)

In [6]:
df = df[df['text'].notna()]
df.shape

(1753, 9)

In [7]:
df = df[df['Categoria'].notna()]
df.shape

(1753, 9)

In [8]:
df.groupby('Categoria')['text'].count().sort_values(ascending=False)

Categoria
Alimentación             567
No factura               527
Grande superficie        330
Salud y Bienestar        139
Moda                      60
Transporte                33
Educación                 29
Hogar                     29
Entretenimiento           13
Mascotas                  13
Compromisos Bancarios      7
Tecnología                 4
Impuestos                  2
Name: text, dtype: int64

In [9]:
dataframe=df[(df['Categoria'] == 'Alimentación')|
          (df['Categoria'] == 'No factura')|
          (df['Categoria'] == 'Grande superficie')|
          (df['Categoria'] == 'Salud y Bienestar')|
          (df['Categoria'] == 'Moda')]

In [10]:
dataframe.groupby('Categoria')['text'].count().sort_values(ascending=False)

Categoria
Alimentación         567
No factura           527
Grande superficie    330
Salud y Bienestar    139
Moda                  60
Name: text, dtype: int64

# Train/Test split

Primero, realizaremos la división del dataset entre las bases de Test y las bases de Train, con el fin de evitar Data Leakage.

In [11]:
# importamos los paquetes requeridos
from sklearn.model_selection import train_test_split

In [12]:
#realizamos la separación, seleccionando que el 20% de los datos sean parte del set de test:
X_train, X_test, y_train, y_test = train_test_split(dataframe.text, dataframe.Categoria, test_size = 0.30, stratify = dataframe.Categoria, random_state = 42)

Para trabajar con texto tenemos que realizar una serie de pasos antes de poder entrenar un modelo. Es decir, buscar la forma de convertir el texto a una representación numérica que pueda ser interpretable por los algoritmos de clasificación. 
Para ello vamos a realizar una serie de pasos.
- Tokenizar el texto
- Convertir a vectores de términos/documentos
- Aplicar tfidf

Es importante destacar que el `fit` debe hacerse sobre el conjunto de `train` y no sobre el total, ya que `tfidf` tiene en cuenta la frecuencia de aparición de las palabras respecto al total del conjunto. Una vez que usamos el `fit` con el conjunto de entrenamiento podemos aplicar la transformación al conjunto de `test`. 

## Pre- Procesamiento

In [13]:
X_train = X_train.astype(str)

In [14]:
def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('spanish')]
    #stem_words=[stemmer.stem(w) for w in filtered_words]
    #lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)

In [15]:
X_train = [preprocess(item) for item in X_train]

Sanity check - debemos obtener un ejemplo de un texto "limpio":

In [16]:
print(X_train[15])
print(len(X_train))

sfarma are drogas fragua nit sparma droguerias cipriano calle san cipriano tel aut numeracion fac regimen comun tarifa tarifa regimen regimen comun obs regimen grandes contribuyentes fecha emision feoha vigencia prefilo consecutivos factura venta mostrador fecha hora caja ciudad bogota vendedor daniela alejandra carreno castellano nombre cliente primer apellido generico segundo apellido kkkkkkk cedula nit direccion calle telefono producto cant pvp dtd imp valor buprofend metocarbamol tabletas pres und levotironina tes removedor lander vitaminae mad schickquattro uds pres fraccion und paleta tosh pasion palet apolet surtica betametasona cpema nitazoxanda tbs crema corega sulfato magnesia drogam melterios pastillas masticables sbs pres sobri und noraver grpabebidanoche sbs vich vaporub ups pres fraccion unid flumuc age mes sbs pres fraccion und pax dia naranja ses pres fraccion xiund tapaboca termosellado biokemical soul pres fraccion lund paletaje surtida labinpina tabletas pres und boo

## Aplicar TFIDF

In [17]:
# los pasos necesarios para vectorizar los conjuntos de entrenamiento y testeo en este celda

from sklearn.feature_extraction.text import TfidfVectorizer

serie = pd.Series( X_train )

vectorizer = TfidfVectorizer()
train_tfidf_matrix = vectorizer.fit_transform(serie)
test_tfidf_matrix = vectorizer.transform(X_test)

In [ ]:
import joblib

filename = "TFIDF-vectorizer.joblib"
joblib.dump(vectorizer, filename)

In [18]:
X_train = train_tfidf_matrix.todense()
X_test = test_tfidf_matrix.todense()

print( "X_train shape: " + str(X_train.shape))
print( "X_test shape: " + str(X_test.shape ))
print( "y_train shape: " + str(y_train.shape )) 
print( "y_test shape: " + str(y_test.shape ))

X_train shape: (1136, 17756)
X_test shape: (487, 17756)
y_train shape: (1136,)
y_test shape: (487,)


In [19]:
y_train 

1182           No factura
246          Alimentación
1092    Grande superficie
107     Grande superficie
1065         Alimentación
              ...        
377     Grande superficie
1170         Alimentación
1723         Alimentación
539     Grande superficie
986          Alimentación
Name: Categoria, Length: 1136, dtype: object

In [20]:
X_train[5][0]

matrix([[0., 0., 0., ..., 0., 0., 0.]])